In [ ]:
import requests
from bs4 import BeautifulSoup
from langchain.chat_models import init_chat_model
from langchain.tools import tool
from langchain.agents import create_agent
from langchain_community.utilities import WikipediaAPIWrapper, DuckDuckGoSearchAPIWrapper


llm = init_chat_model(model="openai:gpt-5-nano")


@tool
def wikipedia_search(query: str) -> str:
    """
    Search Wikipedia and return search results.
    """
    wiki = WikipediaAPIWrapper()
    return wiki.run(query=query)


@tool
def duckduckgo_search(query: str) -> str:
    """
    Search DuckDuckGo and return search results.
    """
    ddgs = DuckDuckGoSearchAPIWrapper()
    return ddgs.run(query=query)


@tool
def scrape_website(url: str) -> str:
    """
    Scrape visible text content from a website URL.
    """
    try:
        headers = {"User-Agent": "MyAgent/0.1"}    
        response = requests.get(url, headers=headers, timeout=10)
        soup = BeautifulSoup(response.content, "html.parser")

        for tag in soup(["script", "style", "noscript", "header", "footer"]):
            tag.decompose()

        text = soup.get_text(separator=" ", strip=True)
        return text[:5000]
    except requests.RequestException as e:
        return f"[Error] Failed to access {url}: {e}"


@tool
def save_research_to_txt(content: str, filename: str = "research_xz_backdoor.txt") -> None:
    """
    Save research content to a .txt file.
    """
    with open(filename, "w", encoding="utf-8") as f:
        f.write(content)    
    

agent = create_agent(
    model=llm,
    tools=[
        wikipedia_search, 
        duckduckgo_search, 
        scrape_website, 
        save_research_to_txt,
    ],
    system_prompt="""
    You are a research AI agent.

    Your workflow:
    1. Search in Wikipedia.
    2. Search in DuckDuckGo.
    3. if you find a website in DuckDuckGo you should enter the website and extract its content.
    4. Combine findings into a research report.
    5. You should finish by saving the research to a .txt file.
    6. The report should be saved only once.    

    Always use tools when external information is required.
    If a tool fails, do NOT stop, Skip the content.
    """
)

result = agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "Research about the XZ backdoor"
            }
        ]
    }
)


result["messages"][-1].content

'I’ve gathered information on the XZ Utils backdoor (CVE-2024-3094) from multiple sources and compiled a concise research report. I also saved the full report to a text file as requested.\n\nWhat I found (highlights)\n- The XZ Utils backdoor was introduced in upstream tarballs for xz-utils versions 5.6.0 and 5.6.1 in February 2024. It allowed pre-auth remote code execution via OpenSSH when the attacker possessed a specific Ed448 private key.\n- The compromise was in the tarballs released by the upstream project, not in the Git source code. Distributions that pulled from those tarballs were affected.\n- The vulnerability is tracked as CVE-2024-3094 and scored 10.0 (critical) on the CVSS scale.\n- The incident was publicly disclosed by Andres Freund on March 29, 2024 and prompted extensive security advisories and vendor responses (Debian, Red Hat, etc.). A clean update to 5.6.2 and later releases was issued to mitigate the issue.\n- The backdoor and its analysis have been widely document